# This is an NHS Jobtrac webscraper, it will pull all job details other than hiring manager details (they are censored against webscraping even if this code was changed to allow this)

This is a neutered version of another webscraper, that extracts all hiring manager details, including Mobile numbers, Names, emails ect. This will not be published due to the nature of it.


When code is run, input the below for it to generate an example of how it works.
https://www.nhsjobs.com/job_list?JobSearch_q=&JobSearch_d=&JobSearch_g=&JobSearch_re=_POST&JobSearch_re_0=1&JobSearch_re_1=1-_-_-&JobSearch_re_2=1-_-_--_-_-&JobSearch_Submit=Search&_tr=JobSearch&_ts=1336&_pg=

In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'relative/path/to/package'))

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse
from lxml import html
import datetime
import tkinter
from tkinter import simpledialog
from tkinter import filedialog

#The below is used for the full version:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
import openpyxl
import time
import random

In [ ]:

base_url = simpledialog.askstring("Input", "Enter the NHS Jobs URL (without page number):")
base_url += "pg="  #add "&page=" part to URL
num_pages_str = simpledialog.askstring("Input", "Enter the number of pages you want to extract:")
num_pages = int(num_pages_str) if num_pages_str is not None else 1
# Define the XPath expressions for data extraction
xpaths = {
    "employer_name": '//*[@id="hj-job-summary"]/div/div/div/dl[2]/dd[1]',
    "job_title": '//*[@id="hj-enquiry-html"]/div/dl/dd[2]',
    'area recruiting for': '//*[@id="hj-job-summary"]/div/div/div/dl[1]/dd[1]',
    'job recruiting for': '//*[@id="hj-job"]/div/div[1]/h2',
    'recruitment close date': '//*[@id="hj-job-summary"]/div/div/div/dl[2]/dd[7]',
    'contract': '//*[@id="hj-job-summary"]/div/div/div/dl[1]/dd[3]',
    'reference': '//*[@id="hj-job-summary"]/div/div/div/dl[1]/dd[5]',
    'Hours': '//*[@id="hj-job-summary"]/div/div/div/dl[1]/dd[4]',
    'address line 1': '//*[@id="hj-job-summary"]/div/div/div/dl[2]/dd[3]',
    'town': '//*[@id="hj-job-summary"]/div/div/div/dl[2]/dd[4]',
    'salary': '//*[@id="hj-job-summary"]/div/div/div/dl[2]/dd[5]',
    'Band': '//*[@id="hj-job-summary"]/div/div/div/dl[1]/dd[2]',
    'personal spec': '//*[@id="hj-job-role-requirement"]/div',
    'job description': '//*[@id="hj-job-advert"]'


}

In [ ]:
dataframes = [] # initialise empty dataframe

In [ ]:

session = requests.Session()
session.max_redirects = 2  # Sets a limit on the number of redirects
# Goes through url, adds page numbers and extracts xpath data from all ads on those pages
for page in range(1, num_pages + 1):
    url = base_url + str(page)
    response = session.get(url)
    if response.status_code == 200:
        print(f" successfully located" +url + '/'+num_pages_str)
        soup = BeautifulSoup(response.text, "html.parser")
        job_adverts = soup.find_all("a", href=True, attrs={"href": "/job/UK/"})

        for advert in job_adverts:
            advert_relative_url = advert["href"]
            if "/job/UK/" in advert_relative_url:
                advert_url = urljoin(base_url, advert_relative_url)
                url_parts = urlparse(advert_url)
                if url_parts.scheme:
                    try:
                        advert_response = session.get(advert_url)
                        advert_response.raise_for_status()
                        advert_tree = html.fromstring(advert_response.text)
                        advert_data = {}
                        for key, xpath in xpaths.items():
                            elements = advert_tree.xpath(xpath)
                            if elements:
                                advert_data[key] = elements[0].text_content()
                            else:
                                print(f"No specified XPath elements detected for {key}")
                        dataframes.append(pd.DataFrame(advert_data, index=[0]))
                    except requests.exceptions.HTTPError as http_err:
                        print(f"HTTP error occurred for URL {advert_url}: {http_err}")
                    except Exception as err:
                        print(f"An error occurred for URL {advert_url}: {err}")
                else:
                    print(f"Invalid URL: {advert_url}")

In [ ]:

# Concatenate dataframe
if dataframes:
    final_dataframe = pd.concat(dataframes, ignore_index=True)
    # Setup the root Tkinter window
    root = tkinter.Tk()
    root.withdraw()  # Hide the root window
    # Ask the user to select the save location and filename using file explorer
    current_datetime = datetime.datetime.now()
    default_filename = f"job_adverts_{current_datetime.strftime('%Y%m%d_%H%M%S')}.xlsx"
    file_path = filedialog.asksaveasfilename(
        initialdir=os.path.expanduser("~/Desktop"),  # Set initial directory to the Desktop
        title="Select file",
        filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")),
        defaultextension=".xlsx",
        initialfile=default_filename
    )

In [1]:

    # Check if a file path was provided (i.e., user didn't cancel the dialog)
    if file_path:
        # Save the DataFrame to an Excel file at the specified path
        final_dataframe.to_excel(file_path, index=False)
        print(f"File saved successfully at {file_path}")
    else:
        print("File save cancelled.")
else:
    simpledialog.messagebox.showinfo("Completion", "No data extracted.")


TypeError: unsupported operand type(s) for +=: 'NoneType' and 'str'